# k.p example of using the Lowdin perturbation

This example requires [semicon](https://gitlab.kwant-project.org/semicon/semicon) to be installed.

It should be as easy as 
```
pip install git+https://gitlab.kwant-project.org/r-j-skolasinski/semicon.git
```

In [ ]:
try:
    import semicon
except ImportError:
    print("Semicon should be installed to run this notebook.")

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 

In [ ]:
import kwant
import semicon

import numpy as np
import scipy.linalg as la

import sympy
sympy.init_printing(print_builtin=False)

import matplotlib.pyplot as plt
%matplotlib inline

# Prepare model

In [ ]:
widths = [5, 5, 5, 5]
gamma_0 = 1.0

grid_spacing = 0.5
shape = lambda site: 0 - grid_spacing / 2 < site.pos[0] < sum(widths)

kpham = semicon.models.foreman('z')

AlSb = semicon.parameters.bulk('lawaetz', 'AlSb', gamma_0, valence_band_offset=.18)
InAs = semicon.parameters.bulk('lawaetz', 'InAs', gamma_0)
GaSb = semicon.parameters.bulk('lawaetz', 'GaSb', gamma_0, valence_band_offset=.56)

params, walls = semicon.parameters.two_deg(
    parameters = [AlSb, InAs, GaSb, AlSb],
    widths = widths,
    grid_spacing=grid_spacing,
    extra_constants=semicon.parameters.constants,
)

# Solve it exactly

In [ ]:
template = kwant.continuum.discretize(str(kpham), coords='z', grid_spacing=grid_spacing)

syst = kwant.Builder()
syst.fill(template, shape, (0, ))
syst = syst.finalized()

N = len(syst.sites)

In [ ]:
momenta = np.linspace(-.4, .4, 101)
energies = []

for k in momenta:
    p = {'k_x': k, 'k_y': 0}
    mat = syst.hamiltonian_submatrix(params={**p, **params})
    energies.append(la.eigvalsh(mat))
    
energies = np.array(energies)

In [ ]:
plt.plot(momenta, 1000 * energies, 'k-');
plt.ylim(420, 820)

# Get effective model 

In [ ]:
from codes.lowdin import prepare_hamiltonian, first_order, second_order_explicit

In [ ]:
%%time
gens = ['k_x', 'k_y']
H0, H1 = prepare_hamiltonian(
    kpham, gens, 'z', grid_spacing, shape, (0,)
)

mat0 = H0.hamiltonian_submatrix(params=params)
mat1 = {k: v.hamiltonian_submatrix(params=params) for k, v in H1.items()}

In [ ]:
%%time
ev, evec = la.eigh(mat0)
indices = list(range(6*N, 6*N+2))

In [ ]:
%%time

def decouple(operator, subspace):
    matrix = subspace.T.conjugate() @ operator @ subspace
    ev, evec = la.eigh(matrix)
    return subspace @ evec

M = np.diag([1/2, -1/2, 3/2, 1/2, -1/2, -3/2, 1/2, -1/2])
bigM = la.block_diag(*[M for i in range(len(syst.sites))])
assert np.allclose(bigM @ mat0 - mat0 @ bigM, np.zeros(mat0.shape))

for i in indices[::2]:
    evec[:, i:i+2] = decouple(bigM, evec[:, i:i+2])

# apply "smart" gauge choice to prettify final output
# I have no idea how this works, but it works, and is wonderful.
# It is a pure magic!
for i, v in enumerate(evec.T):
    phase = np.angle(v @ v)
    evec[:, i] = v * np.exp(-1j*phase/2)

In [ ]:
%%time
M1 = first_order(mat1, evec[:, indices])

In [ ]:
%%time
M2 = second_order_explicit(mat1, indices, ev, evec)

In [ ]:
for k, v in M1.items():
    print(k)
    print(v)
    print()

In [ ]:
for k, v in M2.items():
    print(k)
    print(v)
    print()

In [ ]:
def sympify_perturbation(M1=None, M2=None, decimals=12):
    terms = []
    terms += [(k, v) for k, v in M1.items() if M1 is not None]
    terms += [(k[0] * k[1], v) for k, v in M2.items() if M2 is not None]

    if len(terms) == 0:
        raise ValueError("At least one of 'M1' or 'M2' should contain "
                         "some items.")
    output = []
    for k, v in terms:
        output.append(k * sympy.Matrix(np.round(v, decimals)))
    return sympy.MatAdd(*output).as_explicit()
        

In [ ]:
sympify_perturbation(M1, M2)